<a href="https://colab.research.google.com/github/datle2403/datle2403/blob/main/hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
work_train= pd.read_csv('work_train.csv')
work_test=pd.read_csv('work_test.csv')

info_train=pd.read_csv('info_train.csv')
info_test= pd.read_csv('info_test.csv')

label_train= pd.read_csv('label_train.csv')
label_test= pd.read_csv('label_test.csv')


In [2]:
!pip install jellyfish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.1 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=70604 sha256=b6b3c5dce569446e525eb23a3ddcc0e0add159cc29e859c636744f3df2be495c
  Stored in directory: /root/.cache/pip/wheels/f1/c7/3c/4c83132de76359e3a429fd09c08995945ca96c5290a41651d3
Successfully built jellyfish


start from here

In [3]:
from algo import isin, isin_job, process, process_hcm, process_hn, process_job

In [4]:
work_train_exp=work_train.copy()
work_train_exp=work_train_exp.drop_duplicates(subset=['id_bh'])

work_train_exp= work_train_exp.merge(info_train, right_on='id_bh',left_on='id_bh')

work_train_exp=work_train_exp.drop(['id','address_y'],axis=1)
work_train_exp['address_x']=work_train_exp['address_x'].str.strip().str.lower()

work_train_exp['address_x']=work_train_exp['address_x'].map(process_hcm).map(process_hn).map(process).map(isin)
work_train_exp['address_x']=work_train_exp['address_x'].fillna('việt nam')
#work_train_exp= work_train_exp.dropna(subset=['address_x'])




In [5]:
### for duration
work_train_exp['duration']=work_train_exp.to_date - work_train_exp.from_date
work_train_exp=work_train_exp.drop(['from_date','to_date'],axis=1)

In [6]:
doc_job='Công nhân, Nhân viên, Giáo viên, Giám đốc, Kế toán, Chuyên viên, Lái xe, Bảo vệ, Lao động, Kỹ sư, Phó giám đốc, Cán bộ, Kỹ thuật viên, Thủ quỹ, Thủ kho, Y sỹ, Quản lý, Kỹ thuật, Giảng viên, Kỹ sư, Kế toán, Điều dưỡng, Trưởng phòng, Giao dịch viên, Chuyên gia, Bán hàng, Phóng viên, Tạp vụ, Cán sự, Tài xế, Cán bộ, Bác sỹ, Văn thư, Thợ may, Bí thư, Tổ trưởng, Lập trình viên, Dược sỹ, Bộ đội, Kiến trúc sư, Phiên dịch, Thợ hàn, Thư ký, công an, địa chất'
doc_job=doc_job.split(', ')
doc_job=list(map(lambda x: x.lower(), doc_job))

In [7]:
### for job/role
work_train_exp['job/role']=work_train_exp['job/role'].str.strip().str.lower()
work_train_exp['job/role']=work_train_exp['job/role'].map(process_job).map(isin_job)

work_train_exp['job/role']=work_train_exp['job/role'].fillna('công nhân')
#work_train_exp= work_train_exp.dropna(subset=['job/role'])

In [8]:
work_train_exp.to_csv('result.csv')

In [9]:
work_train_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27502 entries, 0 to 27501
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_bh          27502 non-null  int64  
 1   id_management  27502 non-null  int64  
 2   id_office      27489 non-null  object 
 3   company_type   27502 non-null  int64  
 4   job/role       27502 non-null  object 
 5   employee_lv    27499 non-null  float64
 6   address_x      27502 non-null  object 
 7   bithYear       27502 non-null  int64  
 8   gender         27502 non-null  object 
 9   duration       27502 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 2.3+ MB


In [10]:
work_train_exp=work_train_exp.drop(['id_management','id_office'], axis=1)

In [11]:
work_train_exp.to_csv('result.csv')

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
missing_value_pipeline=Pipeline([
    ('missing_value',SimpleImputer(strategy='most_frequent', missing_values=np.nan))
])

In [13]:
train_ds= work_train_exp.merge(label_train,left_on='id_bh',right_on='id_bh')

In [14]:
train_ds.head()

,id_bh,company_type,job/role,employee_lv,address_x,bithYear,gender,duration,label
0,113039360,-1,giám đốc,7.0,hồ chí minh,1971,MALE,21100,4
1,116074930,-1,nhân viên,7.0,hồ chí minh,1993,FEMALE,600,2
2,203060233,-1,kế toán,1.0,việt nam,1977,MALE,20600,4
3,131373210,-1,nhân viên,8.0,hồ chí minh,1996,FEMALE,0,2
4,198079441,-1,công nhân,-1.0,việt nam,1971,MALE,160500,2


In [29]:
X, y= train_ds.iloc[:,1:8], train_ds.iloc[:,-1]

In [30]:
y.unique()

array([4, 2, 5, 6, 3, 1, 7])

In [31]:
ohe_col=['company_type','job/role','address_x','gender']
standard_col=['duration','employee_lv']
bucketized_col=['bithYear']
missing_value_col=['employee_lv']

In [32]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from keras.utils import to_categorical
pipeline= ColumnTransformer([
    ('missing_value', missing_value_pipeline,missing_value_col),
    ('ohe',OneHotEncoder(),ohe_col),
    ('standard', StandardScaler(), standard_col),
    ('bucket', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform'), bucketized_col)
],remainder='passthrough')

In [33]:
ohe= OneHotEncoder()
y=ohe.fit_transform(y.values.reshape(-1,1)).toarray()

In [34]:
X=pipeline.fit_transform(X).toarray()

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y,train_size=0.9, stratify=y)

In [36]:
X_train.shape

(24751, 115)

In [37]:
X_test.shape

(2751, 115)

In [38]:
y_train[0]

array([0., 0., 0., 1., 0., 0., 0.])

In [39]:
X_train[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.  

In [41]:
import tensorflow as tf
from tensorflow.keras import layers

input= tf.keras.Input((115,))

h1=layers.Dense(units=16,activation='relu')(input)
h2=layers.Dense(units=16,activation='relu')(h1)
h3=layers.Dense(units=16,activation='relu')(h2)

output= layers.Dense(units=7, activation='softmax')(h3)

model= tf.keras.Model(inputs= input, outputs= output)

In [42]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 115)]             0         
                                                                 
 dense_4 (Dense)             (None, 16)                1856      
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 16)                272       
                                                                 
 dense_7 (Dense)             (None, 7)                 119       
                                                                 
Total params: 2,519
Trainable params: 2,519
Non-trainable params: 0
_________________________________________________________________


In [43]:
opt= tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'] , optimizer=opt)

In [ ]:
hist= model.fit(x=X_train, y=y_train, batch_size=100, validation_data=(X_test,y_test), verbose=1,epochs=200)

Epoch 1/200
248/248 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 2/200
248/248 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 3/200
248/248 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 4/200
248/248 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 5/200
248/248 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 6/200
248/248 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 7/200
248/248 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.0044 - val_loss: nan - val_accuracy: 0.0044
Epoch 8/200
248/248 [====================

In [45]:
history= hist.history
history

{'loss': [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan